
# Clean data

## Imports

In [1]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np

## Load  original data

In [2]:
df = pd.read_csv('../data/NCDB_1999_to_2017.csv')
%time

/Users/sowerre/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,2,5,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


CPU times: user 4 µs, sys: 16 µs, total: 20 µs
Wall time: 23.1 µs


In [3]:
df.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE
0,1999,1,1,20,2,02,34,UU,1,5,...,06,1990,01,M,41,11,1,UU,1,752
1,1999,1,1,20,2,02,34,UU,1,5,...,01,1987,01,M,19,11,1,UU,1,752
2,1999,1,1,20,2,02,34,UU,1,5,...,01,1987,02,F,20,13,2,02,2,752
3,1999,1,1,08,2,01,01,UU,5,3,...,01,1986,01,M,46,11,1,UU,1,753
4,1999,1,1,08,2,01,01,UU,5,3,...,NN,NNNN,01,M,05,99,2,UU,3,753


In the data, each of the attribute's values with U, UU, UUU, and UUUU represent unknown.

X, XX, XXX, and XXXX represent Jurisdiction does not provide this data element. 

Q, QQ, QQQ, QQQQ represent Choice is other than the preceding values.  

N, NN, NNN, NNNN represent Data element is not applicable.

Please see the explanatory docx file at [National collision database](https://open.canada.ca/data/en/dataset/1eb9eba7-71d1-4b30-9fb1-30cbdab7e63a) for a full explanation of the attributes and their values.

In [4]:
df.shape # Dimension of the data

(6772563, 23)

## Select features for time series analysis

We are interested in univariate time series of the monthly fatality on public roads in Canada. For this reason, we will select few attrivutes which include the year of collision (C_YEAR), the month of collision (C_MNTH), and the fatality class (C_SEV).

In [5]:
df_selected = df[['C_YEAR','C_MNTH','C_SEV']] # Select the features

In [6]:
df_selected.to_csv('../data/NCDB_time_series.csv', index =False) # Save as csv file

In [7]:
# reload data
df = pd.read_csv('../data/NCDB_time_series.csv')
%time

/Users/sowerre/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs


In [8]:
df.head()

,C_YEAR,C_MNTH,C_SEV
0,1999,1,2
1,1999,1,2
2,1999,1,2
3,1999,1,2
4,1999,1,2


##  Replace non-numeric values with numpy nan

In [9]:
letter_list = ['U', 'UU', 'UUU', 'UUUU', 'Q', 'QQ', 'QQQ', 'QQQQ', 
               'N', 'NN', 'NNN', 'NNNN', 'X', 'XX', 'XXX', 'XXXX']
df = df.replace(letter_list, np.nan)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6772563 entries, 0 to 6772562
Data columns (total 3 columns):
C_YEAR    int64
C_MNTH    object
C_SEV     int64
dtypes: int64(2), object(1)
memory usage: 155.0+ MB


##  Missing values

In [10]:
# Percentage of missing values
missing_values = (df.isnull().sum()/df.shape[0])*100

# Create a dataframe of missing values
Missing_df =pd.DataFrame({'Variable': missing_values.index,
                          'Percentage of missing values': missing_values.values})
Missing_df

,Variable,Percentage of missing values
0,C_YEAR,0.000000
1,C_MNTH,0.006275
2,C_SEV,0.000000


##  Month of the year

In [11]:
df_selected.C_MNTH.unique() # Get the unique values in C_MNTH

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, '12', 'UU', '01', '02',
       '11'], dtype=object)

In [12]:
df_selected.C_MNTH = pd.to_numeric(df_selected['C_MNTH'], errors = 'coerce') # Convert to numerical values

In [13]:
df_selected.C_MNTH.unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., nan])

In [14]:
np.dtype(df_selected.C_MNTH) # Data type

dtype('float64')

## Fatality class

The fatality class is encoded as integers, where 2 represents non-fatal collisions and 1 represents fatal collisions. We will recode this as binaries (0 and 1) with 0 representing the non-fatal collisions. Our main objective is to analyze the time series of the monthly fatality

In [15]:
df_selected.C_SEV.unique()

array([2, 1])

In [16]:
df_selected.C_SEV.value_counts()

2    6660073
1     112490
Name: C_SEV, dtype: int64

In [17]:
df_selected['C_SEV']= df_selected['C_SEV'].replace(2, 0) # Replace 2 with 0

In [18]:
df_selected.C_SEV.unique()

array([0, 1])

#  Create a Time Series Data and Analyze

The first step in time series analysis is to put the data in a time series format. Below we use pandas pivot_table function to calculate the sum of fatality indexed by the year of collision with the columns as the month of collision. We then stack the result and form a time series data set

In [19]:
table_year= df_selected.pivot_table('C_SEV', index = 'C_YEAR', columns = 'C_MNTH', aggfunc = 'sum')
table_month= df_selected.pivot_table('C_SEV', index = 'C_MNTH', columns = 'C_YEAR', aggfunc = 'sum')

In [20]:
table_year.head()

C_MNTH,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
C_YEAR,,,,,,,,,,,,
1999,433,482,618,438,637,645,763,730,789,648,659,756
2000,566,437,419,471,542,596,740,712,734,617,537,640
2001,408,458,451,448,498,656,594,780,588,571,573,693
2002,510,460,446,418,530,625,714,743,668,647,600,783
2003,540,347,501,430,522,605,808,760,604,615,557,581


In [21]:
# table_month.plot(figsize=(20,8), marker='o')
# plt.legend(loc = 'upper left')

In [22]:
ts = table_year.stack() # Time series data set

In [23]:
ts.head(15)

C_YEAR  C_MNTH
1999    1.0       433
        2.0       482
        3.0       618
        4.0       438
        5.0       637
        6.0       645
        7.0       763
        8.0       730
        9.0       789
        10.0      648
        11.0      659
        12.0      756
2000    1.0       566
        2.0       437
        3.0       419
dtype: int64

In [24]:
year = ts.index.get_level_values(0).values
month = ts.index.get_level_values(1).values

In [25]:
ts.index = pd.PeriodIndex(year=year, month=month, freq='M')

In [26]:
df = pd.DataFrame({'year': ts.index, 'fatalities':ts.values})

In [27]:
df.head()

,year,fatalities
0,1999-01,433
1,1999-02,482
2,1999-03,618
3,1999-04,438
4,1999-05,637


In [28]:
#save data
df.to_csv('../data/collision_ts.csv', index = False)